# EFM Large-Scale Structure (LSS) Parameter Sweep (Dimensionless, A100 Focused)

This notebook performs a systematic parameter sweep for key dimensionless coefficients in the Eholoko Fluxon Model (EFM) Nonlinear Klein-Gordon (NLKG) equation. The objective is to identify optimal parameter combinations that robustly produce the characteristic Large-Scale Structure (LSS) clustering scales (147 Mpc and 628 Mpc) predicted by EFM.

Building upon previous LSS simulations (e.g., LSS_N450_T200000_..._Seeded) that demonstrated the amplification of seeded modes, this sweep focuses on refining the `g_sim` (cubic nonlinearity) and `k_efm_gravity_coupling` (self-gravity strength) parameters. These parameters are crucial for driving the 'Fluxonic Clustering' mechanism, where the scalar field self-organizes into cosmic structures without the need for dark matter.

Each simulation in the sweep will operate in dimensionless units on an A100 GPU, ensuring consistency with EFM's fundamental theoretical framework.

## EFM Theoretical Grounding:

The simulation uses the dimensionless NLKG equation from the 'Ehokolo Fluxon Model: Unifying Cosmic Structure, Non-Gaussianity, and Gravitational Waves Across Scales' paper [1]. The parameters being swept directly influence the field's self-interaction and self-gravity, which are the driving forces behind EFM's structure formation.


## Google Drive Setup (for Colab)

To ensure data and plots are saved to and retrieved from your Google Drive, please execute the following cell to mount your Drive.


In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not in Google Colab environment. Skipping Google Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")


In [ ]:
import os
import torch
import torch.nn as nn
import gc
import psutil
from tqdm.notebook import tqdm
import numpy as np
import time
from datetime import datetime
from scipy.fft import fftn, fftfreq, ifftn
import torch.nn.functional as F
import torch.amp as amp
import matplotlib.pyplot as plt
import glob
import scipy.signal  # For find_peaks

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch version: {torch.__version__}")
num_gpus_available = torch.cuda.device_count()
available_devices_list = [torch.device(f'cuda:{i}') for i in range(num_gpus_available)]
print(f"Number of GPUs available: {num_gpus_available}, Available devices: {available_devices_list}")
if num_gpus_available > 0:
    current_gpu_device = torch.device('cuda:0')
    print(f"Using GPU 0: {torch.cuda.get_device_name(current_gpu_device)}, VRAM: {torch.cuda.get_device_properties(current_gpu_device).total_memory / 1e9:.2f} GB")
else:
    current_gpu_device = torch.device('cpu')
    print("No GPU available, running on CPU. Performance may be limited.")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Define paths for checkpoints and data/plots
checkpoint_path_lss_sweep = '/content/drive/My Drive/EFM_Simulations/checkpoints/LSS_DIMLESS_A100_Sweep/'
data_path_lss_sweep = '/content/drive/My Drive/EFM_Simulations/data/LSS_DIMLESS_A100_Sweep/'
os.makedirs(checkpoint_path_lss_sweep, exist_ok=True)
os.makedirs(data_path_lss_sweep, exist_ok=True)
print(f"LSS Sweep Checkpoints will be saved to: {checkpoint_path_lss_sweep}")
print(f"LSS Sweep Data/Plots will be saved to: {data_path_lss_sweep}")


## Parameter Sweep Configuration

This configuration defines the ranges for the parameter sweep. The core simulation parameters are fixed (similar to LSS_v4, but N/T_steps adjusted for faster individual runs).

**Parameters for Sweep:**
*   `g_sim`: Cubic nonlinearity coefficient [1, Section 2].
*   `k_efm_gravity_coupling`: EFM self-gravity coupling [1, Section 2].

**Fixed Core Parameters (Dimensionless):**
*   `N`: Grid size. Set to `250` for faster sweep points.
*   `T_steps`: Total timesteps. Set to `50000` for faster sweep points.
*   `m_sim_unit_inv`: Mass term. Fixed at `1.0` [1].
*   `alpha_sim`: State parameter. Fixed at `0.7` for S/T state [1].
*   `delta_sim`: Dissipation term. Fixed at `0.0002` [1].
*   `seeded_perturbation_amplitude`: Amplitude of seeded modes. Fixed at `1.0e-3`.
*   `background_noise_amplitude`: Amplitude of general random noise. Fixed at `1.0e-6`.
*   `k_seed_primary`, `k_seed_secondary`: Wavenumbers for seeded modes. Fixed as per previous optimal LSS runs.

This sweep aims to systematically explore the interplay of `g_sim` and `k_efm_gravity_coupling` to find the most prominent clustering features.

In [ ]:
# --- Fixed Core Simulation Parameters (Dimensionless) ---
base_config = {
    'N': 250,  # Grid size for sweep runs (smaller for faster iterations)
    'L_sim_unit': 10.0,
    'c_sim_unit': 1.0,
    'dt_cfl_factor': 0.001,
    'T_steps': 50000,  # Timesteps for sweep runs (shorter for faster iterations)
    'm_sim_unit_inv': 1.0,
    'eta_sim': 0.01,
    'G_sim_unit': 1.0,
    'alpha_sim': 0.7,
    'delta_sim': 0.0002,
    'seeded_perturbation_amplitude': 1.0e-3,
    'background_noise_amplitude': 1.0e-6,
    'k_seed_primary': 2 * np.pi / (10.0 / 2.0),  # L/2 wavelength
    'k_seed_secondary': 2 * np.pi / (10.0 / 8.0),  # L/8 wavelength (targets 1.25 dimless for 157 Mpc)
    'history_every_n_steps': 1000,
    'checkpoint_every_n_steps': 5000,
}
base_config['dx_sim_unit'] = base_config['L_sim_unit'] / base_config['N']
base_config['dt_sim_unit'] = base_config['dt_cfl_factor'] * base_config['dx_sim_unit'] / base_config['c_sim_unit']

# --- Parameters to Sweep ---
g_values = [0.05, 0.1, 0.5]  # Values for g_sim (cubic nonlinearity)
k_gravity_values = [0.001, 0.005, 0.01]  # Values for k_efm_gravity_coupling

sweep_params = []
for g in g_values:
    for k_g in k_gravity_values:
        # Create a mutable copy of base_config for each sweep point
        config = base_config.copy()
        config['g_sim'] = g
        config['k_efm_gravity_coupling'] = k_g
        config['run_id'] = (
            f"LSS_Sweep_N{config['N']}_T{config['T_steps']}_" +
            f"g{config['g_sim']:.1e}_k{config['k_efm_gravity_coupling']:.1e}_" +
            f"A100_DIMLESS_Sweep"
        )
        sweep_params.append(config)

print(f"Prepared {len(sweep_params)} sweep configurations.")
for i, p in enumerate(sweep_params):
    print(f"Sweep {i+1}: g={p['g_sim']:.2g}, k_gravity={p['k_efm_gravity_coupling']:.2g}")


## Core Simulation Functions

These functions define the EFM NLKG module, the RK4 time integration, and the energy/density norm calculation. These are identical to the optimized versions used in the main LSS simulation notebook (`lssv20.ipynb`).


In [ ]:
class EFMLSSModule(nn.Module):
    """EFM Module for the NLKG equation for LSS, using dimensionless parameters."""
    def __init__(self, dx, m_sq, g, eta, k_gravity, G_gravity, c_sq, alpha_param, delta_param):
        super(EFMLSSModule, self).__init__()
        self.dx = dx
        self.m_sq = m_sq
        self.g = g
        self.eta = eta
        self.k_gravity = k_gravity
        self.G_gravity = G_gravity
        self.c_sq = c_sq
        self.alpha_param = alpha_param
        self.delta_param = delta_param
        stencil_np = np.array([[[0,0,0],[0,1,0],[0,0,0]],
                               [[0,1,0],[1,-6,1],[0,1,0]],
                               [[0,0,0],[0,1,0],[0,0,0]]], dtype=np.float32)
        self.stencil = torch.from_numpy(stencil_np / (dx**2)).to(torch.float16).view(1, 1, 3, 3, 3)

    def conv_laplacian(self, phi_field):
        stencil_dev = self.stencil.to(phi_field.device, phi_field.dtype)
        phi_reshaped = phi_field.unsqueeze(0).unsqueeze(0)
        phi_padded = F.pad(phi_reshaped, (1,1,1,1,1,1), mode='circular')
        laplacian = F.conv3d(phi_padded, stencil_dev, padding=0)
        return laplacian.squeeze(0).squeeze(0)

    def nlkg_derivative_lss(self, phi, phi_dot):
        lap_phi = self.conv_laplacian(phi)
        potential_force = self.m_sq * phi + self.g * torch.pow(phi, 3) + self.eta * torch.pow(phi, 5)
        grad_phi_x = (torch.roll(phi, shifts=-1, dims=0) - torch.roll(phi, shifts=1, dims=0)) / (2 * self.dx)
        grad_phi_y = (torch.roll(phi, shifts=-1, dims=1) - torch.roll(phi, shifts=1, dims=1)) / (2 * self.dx)
        grad_phi_z = (torch.roll(phi, shifts=-1, dims=2) - torch.roll(phi, shifts=1, dims=2)) / (2 * self.dx)
        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        alpha_term = self.alpha_param * phi * phi_dot * grad_phi_abs_sq
        delta_term = self.delta_param * torch.pow(phi_dot, 2) * phi
        source_gravity = 8.0 * float(np.pi) * self.G_gravity * self.k_gravity * torch.pow(phi, 2)
        phi_ddot = self.c_sq * lap_phi - potential_force + alpha_term + delta_term + source_gravity
        return phi_dot, phi_ddot

def update_phi_rk4_lss(phi_current: torch.Tensor, phi_dot_current: torch.Tensor,
                       dt: float, model_instance: EFMLSSModule) -> tuple[torch.Tensor, torch.Tensor]:
    with amp.autocast(device_type=phi_current.device.type, dtype=torch.float16):
        k1_v, k1_a = model_instance.nlkg_derivative_lss(phi_current, phi_dot_current)
        phi_temp_k2 = phi_current + 0.5 * dt * k1_v
        phi_dot_temp_k2 = phi_dot_current + 0.5 * dt * k1_a
        k2_v, k2_a = model_instance.nlkg_derivative_lss(phi_temp_k2, phi_dot_temp_k2)
        phi_temp_k3 = phi_current + 0.5 * dt * k2_v
        phi_dot_temp_k3 = phi_dot_current + 0.5 * dt * k2_a
        k3_v, k3_a = model_instance.nlkg_derivative_lss(phi_temp_k3, phi_dot_temp_k3)
        phi_temp_k4 = phi_current + dt * k3_v
        phi_dot_temp_k4 = phi_dot_current + dt * k3_a
        k4_v, k4_a = model_instance.nlkg_derivative_lss(phi_temp_k4, phi_dot_temp_k4)
        phi_next = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        phi_dot_next = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)
    del k1_v, k1_a, k2_v, k2_a, k3_v, k3_a, k4_v, k4_a
    del phi_temp_k2, phi_dot_temp_k2, phi_temp_k3, phi_dot_temp_k3, phi_temp_k4, phi_dot_temp_k4
    return phi_next, phi_dot_next

def compute_total_energy_lss(phi: torch.Tensor, phi_dot: torch.Tensor,
                             m_sq_param: float, g_param: float, eta_param: float,
                             dx: float, c_sq_param: float) -> float:
    vol_element = dx**3
    phi_f32 = phi.to(dtype=torch.float32)
    phi_dot_f32 = phi_dot.to(dtype=torch.float32)
    with amp.autocast(device_type=phi.device.type, dtype=torch.float16):
        kinetic_density = 0.5 * torch.pow(phi_dot_f32, 2)
        potential_density = (0.5 * m_sq_param * torch.pow(phi_f32, 2) +
                             0.25 * g_param * torch.pow(phi_f32, 4) +
                             (1.0/6.0) * eta_param * torch.pow(phi_f32, 6))
        grad_phi_x = (torch.roll(phi_f32, shifts=-1, dims=0) - torch.roll(phi_f32, shifts=1, dims=0)) / (2 * dx)
        grad_phi_y = (torch.roll(phi_f32, shifts=-1, dims=1) - torch.roll(phi_f32, shifts=1, dims=1)) / (2 * dx)
        grad_phi_z = (torch.roll(phi_f32, shifts=-1, dims=2) - torch.roll(phi_f32, shifts=1, dims=2)) / (2 * dx)
        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        gradient_energy_density = 0.5 * c_sq_param * grad_phi_abs_sq
        total_energy_current_chunk = torch.sum(kinetic_density + potential_density + gradient_energy_density) * vol_element
    if torch.isnan(total_energy_current_chunk) or torch.isinf(total_energy_current_chunk):
        return float('nan')
    total_energy_val = total_energy_current_chunk.item()
    del phi_f32, phi_dot_f32, kinetic_density, potential_density, gradient_energy_density
    del grad_phi_x, grad_phi_y, grad_phi_z, grad_phi_abs_sq
    gc.collect()
    torch.cuda.empty_cache()
    return total_energy_val

def compute_power_spectrum_lss(phi_cpu_np_array: np.ndarray, k_val_range: list,
                               dx_val_param: float, N_grid_param: int) -> tuple[np.ndarray, np.ndarray]:
    if not isinstance(phi_cpu_np_array, np.ndarray):
        phi_cpu_np_array = phi_cpu_np_array.cpu().numpy()
    phi_fft_transform = fftn(phi_cpu_np_array.astype(np.float32))
    power_spectrum_raw_data = np.abs(phi_fft_transform)**2 / (N_grid_param**6)
    del phi_fft_transform
    gc.collect()
    kx_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    ky_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    kz_coords = fftfreq(N_grid_param, d=dx_val_param) * 2 * np.pi
    kxx_mesh, kyy_mesh, kzz_mesh = np.meshgrid(kx_coords, ky_coords, kz_coords, indexing='ij', sparse=True)
    k_magnitude_values = np.sqrt(kxx_mesh**2 + kyy_mesh**2 + kzz_mesh**2)
    del kxx_mesh, kyy_mesh, kzz_mesh, kx_coords, ky_coords, kz_coords
    gc.collect()
    k_bins_def = np.linspace(k_val_range[0], k_val_range[1], 50)
    power_binned_values, _ = np.histogram(
        k_magnitude_values.ravel(), bins=k_bins_def,
        weights=power_spectrum_raw_data.ravel(), density=False
    )
    counts_in_bins, _ = np.histogram(k_magnitude_values.ravel(), bins=k_bins_def)
    power_binned_final = np.divide(power_binned_values, counts_in_bins, out=np.zeros_like(power_binned_values), where=counts_in_bins!=0)
    k_bin_centers_final = (k_bins_def[:-1] + k_bins_def[1:]) / 2
    del k_magnitude_values, power_spectrum_raw_data, counts_in_bins
    gc.collect()
    return k_bin_centers_final, power_binned_final

def compute_correlation_function_lss(phi_cpu_np_array: np.ndarray, dx_val_param: float,
                                     N_grid_param: int, L_box_param: float) -> tuple[np.ndarray, np.ndarray]:
    if not isinstance(phi_cpu_np_array, np.ndarray):
        phi_cpu_np_array = phi_cpu_np_array.cpu().numpy()
    phi_fft_transform = fftn(phi_cpu_np_array.astype(np.float32))
    power_spectrum_raw_data = np.abs(phi_fft_transform)**2
    del phi_fft_transform
    gc.collect()
    correlation_func_raw_data = ifftn(power_spectrum_raw_data).real / (N_grid_param**3)
    del power_spectrum_raw_data
    gc.collect()
    indices_shifted = np.fft.ifftshift(np.arange(N_grid_param)) - (N_grid_param // 2)
    rx_coords = indices_shifted * dx_val_param
    ry_coords = indices_shifted * dx_val_param
    rz_coords = indices_shifted * dx_val_param
    rxx_mesh, ryy_mesh, rzz_mesh = np.meshgrid(rx_coords, ry_coords, rz_coords, indexing='ij', sparse=True)
    r_magnitude_values = np.sqrt(rxx_mesh**2 + ryy_mesh**2 + rzz_mesh**2)
    del rx_coords, ry_coords, rz_coords, rxx_mesh, ryy_mesh, rzz_mesh
    gc.collect()
    r_bins_def = np.linspace(0, L_box_param / 2, 50)
    corr_binned_values, _ = np.histogram(
        r_magnitude_values.ravel(), bins=r_bins_def,
        weights=correlation_func_raw_data.ravel()
    )
    counts_in_bins, _ = np.histogram(r_magnitude_values.ravel(), bins=r_bins_def)
    corr_binned_final = np.divide(corr_binned_values, counts_in_bins, out=np.zeros_like(corr_binned_values), where=counts_in_bins!=0)
    r_bin_centers_final = (r_bins_def[:-1] + r_bins_def[1:]) / 2
    del r_magnitude_values, correlation_func_raw_data, counts_in_bins
    gc.collect()
    return r_bin_centers_final, corr_binned_final


## Simulation Orchestration for Parameter Sweep

This section iterates through the defined sweep parameters, runs the LSS simulation for each combination, and collects the results. To manage output for multiple runs, detailed plots for each run will be saved to Google Drive, and a summary of key metrics (peak locations, fNL) will be printed.


In [ ]:
def run_lss_sweep_simulation(config: dict, device: torch.device):
    print(f"Starting LSS Sweep Run ({config['run_id']}) on {device}...")
    
    torch.manual_seed(42)
    np.random.seed(42)

    # Initial conditions: Seeded sinusoidal perturbations + background noise
    x_coords = np.linspace(-config['L_sim_unit']/2, config['L_sim_unit']/2, config['N'], dtype=np.float32)
    X, Y, Z = np.meshgrid(x_coords, x_coords, x_coords, indexing='ij')
    seeded_modes_field = config['seeded_perturbation_amplitude'] * (
        np.sin(config['k_seed_primary'] * X) +
        np.sin(config['k_seed_secondary'] * Y) +
        np.cos(config['k_seed_primary'] * Z)
    )
    random_background_noise = config['background_noise_amplitude'] * (np.random.rand(config['N'], config['N'], config['N']) - 0.5)
    initial_phi_np = seeded_modes_field + random_background_noise
    phi = torch.from_numpy(initial_phi_np.astype(np.float16)).to(device, dtype=torch.float16)
    phi_dot = torch.zeros_like(phi, dtype=torch.float16, device=device)

    efm_model = EFMLSSModule(
        dx=config['dx_sim_unit'], m_sq=config['m_sim_unit_inv']**2, g=config['g_sim'], eta=config['eta_sim'],
        k_gravity=config['k_efm_gravity_coupling'], G_gravity=config['G_sim_unit'], c_sq=config['c_sim_unit']**2,
        alpha_param=config['alpha_sim'], delta_param=config['delta_sim']
    ).to(device)
    efm_model.eval()

    # Removed history lists to optimize memory for many runs

    sim_start_time = time.time()
    numerical_error = False

    for t_step in tqdm(range(config['T_steps']), desc=f"Sweep Run ({config['run_id']})"):
        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)) or \
           torch.any(torch.isinf(phi_dot)) or torch.any(torch.isnan(phi_dot)):
            print(f"\nERROR: NaN/Inf detected in fields at step {t_step + 1}! Stopping run {config['run_id']}.\n")
            numerical_error = True
            break
        phi, phi_dot = update_phi_rk4_lss(phi, phi_dot, config['dt_sim_unit'], efm_model)

    sim_duration = time.time() - sim_start_time
    print(f"Sweep Run {config['run_id']} finished in {sim_duration:.2f} seconds. Error: {numerical_error}")

    # Save final state only for analysis (no history)
    final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_data_filename = os.path.join(data_path_lss_sweep, f"SWEEP_DATA_{config['run_id']}_{final_timestamp}.npz")
    np.savez_compressed(final_data_filename,
                        phi_final_cpu=phi.cpu().numpy(),
                        config_lss=config,
                        sim_had_numerical_error=numerical_error)
    print(f"Final state saved to {final_data_filename}")

    del phi, phi_dot, efm_model
    gc.collect()
    torch.cuda.empty_cache()

    return final_data_filename


## Analysis and Plotting for Sweep Results

This section defines how to analyze each sweep run's final state for clustering peaks and non-Gaussianity. A summary table will be generated for all sweep points.

In [ ]:
def analyze_and_plot_sweep_result(data_file_path: str, sweep_results_summary: list):
    print(f"Analyzing sweep result from: {data_file_path}")
    try:
        data = np.load(data_file_path, allow_pickle=True)
        phi_final_cpu = data['phi_final_cpu']
        config = data['config_lss'].item()
        sim_had_numerical_error = data['sim_had_numerical_error'].item()

        run_summary = {
            'run_id': config['run_id'],
            'g_sim': config['g_sim'],
            'k_efm_gravity_coupling': config['k_efm_gravity_coupling'],
            'status': 'Error' if sim_had_numerical_error else 'Completed',
            'max_phi_amplitude': np.max(np.abs(phi_final_cpu)),
            'pk_peak_k': np.nan,
            'pk_peak_lambda_sim': np.nan,
            'xi_peak_r': np.nan,
            'fNL_calc': np.nan,
            'physical_primary_lambda': np.nan,
            'physical_secondary_lambda': np.nan,
            'secondary_match_percent': np.nan
        }

        if sim_had_numerical_error:
            sweep_results_summary.append(run_summary)
            return

        # --- Power Spectrum and Correlation Function Analysis (Dimensionless) ---
        k_min_plot_sim = 2 * np.pi / config['L_sim_unit'] * 0.5
        k_max_plot_sim = np.pi / config['dx_sim_unit'] * 0.9
        k_bins_sim, pk_vals_sim = compute_power_spectrum_lss(phi_final_cpu, k_val_range=[k_min_plot_sim, k_max_plot_sim], dx_val_param=config['dx_sim_unit'], N_grid_param=config['N'])
        r_bins_sim, xi_vals_sim = compute_correlation_function_lss(phi_final_cpu, dx_val_param=config['dx_sim_unit'], N_grid_param=config['N'], L_box_param=config['L_sim_unit'])

        # --- Plot P(k) and xi(r) for current run ---
        plt.figure(figsize=(16,6))
        plt.subplot(1,2,1)
        plt.loglog(k_bins_sim, pk_vals_sim)
        plt.title(f"P(k) for g={config['g_sim']:.2g}, k_grav={config['k_efm_gravity_coupling']:.2g}")
        plt.xlabel('k (Dimless)'); plt.ylabel('P(k) (Dimless)'); plt.grid(True, which='both')
        plt.axvline(config['k_seed_primary'], color='orange', linestyle='--', label=f"Seeded k_primary ({config['k_seed_primary']:.2f})")
        plt.axvline(config['k_seed_secondary'], color='purple', linestyle='--', label=f"Seeded k_secondary ({config['k_seed_secondary']:.2f})")
        plt.legend(); plt.xlim([k_min_plot_sim, k_max_plot_sim])

        plt.subplot(1,2,2)
        plt.plot(r_bins_sim, xi_vals_sim)
        plt.title(f"xi(r) for g={config['g_sim']:.2g}, k_grav={config['k_efm_gravity_coupling']:.2g}")
        plt.xlabel('r (Dimless)'); plt.ylabel('xi(r) (Dimless)'); plt.grid(True)
        plt.axhline(0, color='black', linewidth=0.5)
        plt.axvline(config['L_sim_unit'] / 2.0, color='orange', linestyle='--', label=f"Seeded lambda1 ({config['L_sim_unit'] / 2.0:.2f})")
        plt.axvline(config['L_sim_unit'] / 8.0, color='purple', linestyle='--', label=f"Seeded lambda2 ({config['L_sim_unit'] / 8.0:.2f})")
        plt.legend()

        plt.tight_layout()
        plot_filename_obs = os.path.join(data_path_lss_sweep, f"sweep_obs_{config['run_id']}.png")
        plt.savefig(plot_filename_obs)
        plt.close()

        # --- Identify and Quantify Emergent Dimensionless Scales ---
        # Find prominent peaks in P(k)
        pk_peaks, pk_properties = scipy.signal.find_peaks(pk_vals_sim, height=np.mean(pk_vals_sim)*5, distance=5)
        if len(pk_peaks) > 0:
            idx_primary_pk_peak = pk_peaks[np.argmin(np.abs(k_bins_sim[pk_peaks] - config['k_seed_primary']))]
            emergent_k_peak_primary = k_bins_sim[idx_primary_pk_peak]
            if emergent_k_peak_primary > 0:
                emergent_lambda_peak_primary = 2*np.pi / emergent_k_peak_primary
            run_summary['pk_peak_k'] = emergent_k_peak_primary
            run_summary['pk_peak_lambda_sim'] = emergent_lambda_peak_primary

        # Find prominent peaks in xi(r)
        xi_positive_part = xi_vals_sim[r_bins_sim > 0.1]
        r_positive_part = r_bins_sim[r_bins_sim > 0.1]
        xi_peaks_r, xi_properties = scipy.signal.find_peaks(xi_positive_part, height=np.max(xi_positive_part)*0.1, distance=5)
        if len(xi_peaks_r) > 0:
            emergent_r_peak_primary = r_positive_part[xi_peaks_r[0]]
            run_summary['xi_peak_r'] = emergent_r_peak_primary

        # --- Physical Interpretation and Match to EFM Predictions ---
        EFM_PRIMARY_LSS_Mpc = 628.0  # Mpc [1]
        EFM_SECONDARY_LSS_Mpc = 157.0  # Mpc (BAO-like) [1]

        if not np.isnan(run_summary['pk_peak_lambda_sim']) and run_summary['pk_peak_lambda_sim'] > 0:
            scaling_factor_L = EFM_PRIMARY_LSS_Mpc / run_summary['pk_peak_lambda_sim']
            physical_primary_lambda_seeded = (config['L_sim_unit'] / 2.0) * scaling_factor_L
            physical_secondary_lambda_seeded = (config['L_sim_unit'] / 8.0) * scaling_factor_L
            secondary_match_percent = abs((physical_secondary_lambda_seeded - EFM_SECONDARY_LSS_Mpc) / EFM_SECONDARY_LSS_Mpc) * 100
            
            run_summary['physical_primary_lambda'] = physical_primary_lambda_seeded
            run_summary['physical_secondary_lambda'] = physical_secondary_lambda_seeded
            run_summary['secondary_match_percent'] = secondary_match_percent

        # --- Non-Gaussianity Analysis (fNL) ---
        N_grid = config['N']
        dx_val = config['dx_sim_unit']
        rho_final_np_for_fNL = (config['k_efm_gravity_coupling'] * phi_final_cpu**2).astype(np.float32)
        rhok_fft = fftn(rho_final_np_for_fNL)
        kx_coords_f = fftfreq(N_grid, d=dx_val) * 2 * np.pi
        ky_coords_f = fftfreq(N_grid, d=dx_val) * 2 * np.pi
        kz_coords_f = fftfreq(N_grid, d=dx_val) * 2 * np.pi
        k_magnitude_f = np.sqrt(kx_coords_f[:, None, None]**2 + ky_coords_f[None, :, None]**2 + kz_coords_f[None, None, :]**2)
        
        target_k_for_fNL_sim = config['k_seed_primary']  # Use the primary seeded k-mode
        k_tolerance = 0.05 * target_k_for_fNL_sim 
        mask_fNL = (k_magnitude_f > (target_k_for_fNL_sim - k_tolerance)) & (k_magnitude_f < (target_k_for_fNL_sim + k_tolerance))
        
        if not np.any(mask_fNL) and len(pk_peaks) > 0:
            k_tolerance = 0.5 * target_k_for_fNL_sim
            mask_fNL = (k_magnitude_f > (target_k_for_fNL_sim - k_tolerance)) & (k_magnitude_f < (target_k_for_fNL_sim + k_tolerance))
        
        fNL_calculated = np.nan
        if np.any(mask_fNL):
            rhok_fft_f32 = rhok_fft.astype(np.float32)
            B_simplified = np.mean(np.abs(rhok_fft_f32 * np.roll(rhok_fft_f32, shifts=-1, axis=0) * np.roll(rhok_fft_f32, shifts=-1, axis=1))[mask_fNL])
            P_simplified = np.mean(np.abs(rhok_fft_f32)**2)[mask_fNL]
            if P_simplified > 1e-30:
                fNL_calculated = (5/3) * B_simplified / (3 * P_simplified**2)
        run_summary['fNL_calc'] = fNL_calculated
        
        sweep_results_summary.append(run_summary)
        
    except Exception as e:
        print(f"Error analyzing/plotting sweep result for {data_file_path}: {e}")
        import traceback
        traceback.print_exc()
        # Ensure summary reflects error
        run_summary = {
            'run_id': os.path.basename(data_file_path),
            'status': f'Analysis Error: {e}',
            'g_sim': 'N/A',
            'k_efm_gravity_coupling': 'N/A',
            'max_phi_amplitude': 'N/A',
            'pk_peak_k': 'N/A',
            'pk_peak_lambda_sim': 'N/A',
            'xi_peak_r': 'N/A',
            'fNL_calc': 'N/A',
            'physical_primary_lambda': 'N/A',
            'physical_secondary_lambda': 'N/A',
            'secondary_match_percent': 'N/A'
        }
        sweep_results_summary.append(run_summary)

    del phi_final_cpu
    gc.collect()
    torch.cuda.empty_cache()


## Main Orchestration Loop

This is the primary execution block that runs all sweep simulations and performs the analysis.

In [ ]:
if __name__ == '__main__':
    # --- Colab specific setup ---
    # Define paths for checkpoints and data/plots
    checkpoint_path_lss_sweep = '/content/drive/My Drive/EFM_Simulations/checkpoints/LSS_DIMLESS_A100_Sweep/'
    data_path_lss_sweep = '/content/drive/My Drive/EFM_Simulations/data/LSS_DIMLESS_A100_Sweep/'

    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
        os.makedirs(checkpoint_path_lss_sweep, exist_ok=True)
        os.makedirs(data_path_lss_sweep, exist_ok=True)
    except ImportError:
        print("Not in Google Colab environment. Skipping Google Drive mount.")
        checkpoint_path_lss_sweep = './EFM_Simulations/checkpoints/LSS_DIMLESS_A100_Sweep/'
        data_path_lss_sweep = './EFM_Simulations/data/LSS_DIMLESS_A100_Sweep/'
        os.makedirs(checkpoint_path_lss_sweep, exist_ok=True)
        os.makedirs(data_path_lss_sweep, exist_ok=True)
    except Exception as e:
        print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")
        checkpoint_path_lss_sweep = './EFM_Simulations/checkpoints/LSS_DIMLESS_A100_Sweep/'
        data_path_lss_sweep = './EFM_Simulations/data/LSS_DIMLESS_A100_Sweep/'
        os.makedirs(checkpoint_path_lss_sweep, exist_ok=True)
        os.makedirs(data_path_lss_sweep, exist_ok=True)

    print(f"LSS Sweep Checkpoints will be saved to: {checkpoint_path_lss_sweep}")
    print(f"LSS Sweep Data/Plots will be saved to: {data_path_lss_sweep}")

    # --- Prepare sweep configurations (defined in previous cell `code-config-sweep`) ---
    # Ensure base_config, g_values, k_gravity_values, and sweep_params are defined there
    # (Re-run that cell if needed to make sweep_params available)
    
    # --- Explicitly define base_config and sweep_params here for robustness ---
    # This is a copy-paste from the 'code-config-sweep' cell
    base_config = {
        'N': 250,  # Grid size for sweep runs (smaller for faster iterations)
        'L_sim_unit': 10.0,
        'c_sim_unit': 1.0,
        'dt_cfl_factor': 0.001,
        'T_steps': 50000,  # Timesteps for sweep runs (shorter for faster iterations)
        'm_sim_unit_inv': 1.0,
        'eta_sim': 0.01,
        'G_sim_unit': 1.0,
        'alpha_sim': 0.7,
        'delta_sim': 0.0002,
        'seeded_perturbation_amplitude': 1.0e-3,
        'background_noise_amplitude': 1.0e-6,
        'k_seed_primary': 2 * np.pi / (10.0 / 2.0),  # L/2 wavelength
        'k_seed_secondary': 2 * np.pi / (10.0 / 8.0),  # L/8 wavelength (targets 1.25 dimless for 157 Mpc)
        'history_every_n_steps': 1000,
        'checkpoint_every_n_steps': 5000,
    }
    base_config['dx_sim_unit'] = base_config['L_sim_unit'] / base_config['N']
    base_config['dt_sim_unit'] = base_config['dt_cfl_factor'] * base_config['dx_sim_unit'] / base_config['c_sim_unit']

    g_values = [0.05, 0.1, 0.5]
    k_gravity_values = [0.001, 0.005, 0.01]

    sweep_params = []
    for g in g_values:
        for k_g in k_gravity_values:
            config = base_config.copy()
            config['g_sim'] = g
            config['k_efm_gravity_coupling'] = k_g
            config['run_id'] = (
                f"LSS_Sweep_N{config['N']}_T{config['T_steps']}_" +
                f"g{config['g_sim']:.1e}_k{config['k_efm_gravity_coupling']:.1e}_" +
                f"A100_DIMLESS_Sweep"
            )
            sweep_params.append(config)
    # --- End of explicit definition ---

    print(f"Prepared {len(sweep_params)} sweep configurations.")

    # Determine the device
    main_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    sweep_results_summary = []

    for i, config_lss_run_current in enumerate(sweep_params):  # Use a different variable name to avoid conflict
        print(f"\n--- Running Sweep Point {i+1}/{len(sweep_params)}: g={config_lss_run_current['g_sim']:.2g}, k_grav={config_lss_run_current['k_efm_gravity_coupling']:.2g} ---\n")
        
        try:
            # Ensure unique directory for each run's checkpoints if needed, otherwise it's just general sweep dir.
            # For simplicity, saving all sweep data to data_path_lss_sweep directly as SWEEP_DATA_run_id.npz
            # No specific checkpointing directories per run for this sweep.
            pass  # No per-run checkpoint directory setup here to simplify
        except Exception as e_clean:
            print(f"Warning: Could not clean checkpoint directory for {config_lss_run_current['run_id']}: {e_clean}. Proceeding anyway.")

        final_data_file_path = run_lss_sweep_simulation(config_lss_run_current, main_device)

        if final_data_file_path:
            analyze_and_plot_sweep_result(final_data_file_path, sweep_results_summary)
        else:
            run_summary = {
                'run_id': config_lss_run_current['run_id'],
                'status': 'Simulation Failed to Produce File',
                'g_sim': config_lss_run_current['g_sim'],
                'k_efm_gravity_coupling': config_lss_run_current['k_efm_gravity_coupling'],
                'max_phi_amplitude': 'N/A',
                'pk_peak_k': 'N/A',
                'pk_peak_lambda_sim': 'N/A',
                'xi_peak_r': 'N/A',
                'fNL_calc': 'N/A',
                'physical_primary_lambda': 'N/A',
                'physical_secondary_lambda': 'N/A',
                'secondary_match_percent': 'N/A'
            }
            sweep_results_summary.append(run_summary)

    print(f"\n--- Parameter Sweep Summary ({datetime.now().strftime('%Y%m%d_%H%M%S')}) ---\n")
    # Print a formatted table of results
    headers = ["Run ID", "g_sim", "k_grav", "Status", "Max Phi", "PK Peak k", "PK Peak L", "Xi Peak r", "fNL", "Phys L1", "Phys L2", "L2 Match %"]
    
    # Helper to format values for table
    def format_val_for_table(val):
        if isinstance(val, float) and np.isnan(val):
            return 'N/A'
        if isinstance(val, (float, np.float32, np.float64)):
            return f"{val:.2e}"
        return str(val)

    # Print header row
    header_row_str = "{:<30} {:<8} {:<8} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(*headers)
    print(header_row_str)
    print("-" * len(header_row_str))  # Adjust separator length dynamically

    # Print data rows
    for res in sweep_results_summary:
        row_vals = [
            res['run_id'],
            format_val_for_table(res['g_sim']),
            format_val_for_table(res['k_efm_gravity_coupling']),
            res['status'],
            format_val_for_table(res['max_phi_amplitude']),
            format_val_for_table(res['pk_peak_k']),
            format_val_for_table(res['pk_peak_lambda_sim']),
            format_val_for_table(res['xi_peak_r']),
            format_val_for_table(res['fNL_calc']),
            format_val_for_table(res['physical_primary_lambda']),
            format_val_for_table(res['physical_secondary_lambda']),
            format_val_for_table(res['secondary_match_percent'])
        ]
        print("{:<30} {:<8} {:<8} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(*row_vals))
    print("-" * len(header_row_str))
